# 🏟️ Betting System MVP (Improved for Colab)

This notebook helps explore exact score betting using Poisson models and basic ML.
- Predict match scores using baseline stats
- Generate high-odds singles and doubles
- Backtest flat stake strategy
- Ready for Google Colab (install dependencies cell included)

⚡ Improvements added:
- First cell installs all dependencies for Colab
- Cleaner odds simulation (realistic ranges)
- Placeholder for uploading real bookmaker odds CSV
- Clearer outputs and plots


In [ ]:
# ✅ Run this first in Colab
!pip install pandas numpy matplotlib scikit-learn lightgbm requests

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import itertools, random
from datetime import datetime
from math import exp, factorial

## 1. Poisson baseline for score predictions

In [ ]:
def poisson(lmbda, k):
    return exp(-lmbda) * lmbda**k / factorial(k)

def predict_score_probs(avg_home_goals, avg_away_goals, max_goals=5):
    probs = {}
    for i in range(max_goals+1):
        for j in range(max_goals+1):
            p = poisson(avg_home_goals, i) * poisson(avg_away_goals, j)
            probs[(i,j)] = p
    return probs

# Example baseline: 1.5 home goals, 1.2 away goals
probs = predict_score_probs(1.5, 1.2)
sorted_probs = sorted(probs.items(), key=lambda x: x[1], reverse=True)[:10]
sorted_probs

## 2. Simulated bookmaker odds
Here we generate fake odds so we can test logic. Later, you can upload real CSV odds.

In [ ]:
random.seed(42)
odds_data = []
teams = ["Arsenal","Chelsea","Liverpool","Man City","Spurs","United"]
for match_id in range(30):
    home, away = random.sample(teams, 2)
    probs = predict_score_probs(random.uniform(1,2), random.uniform(0.8,1.8))
    for (hg,ag),p in probs.items():
        fair_odds = 1/p if p>0 else 1000
        bookie_odds = fair_odds * random.uniform(0.8,1.2)
        odds_data.append([match_id, home, away, f"{hg}-{ag}", bookie_odds, p])
odds_df = pd.DataFrame(odds_data, columns=["MatchID","Home","Away","Score","Odds","ModelProb"])
odds_df.head()

## 3. Finding high-odds singles and doubles

In [ ]:
singles = odds_df[odds_df["Odds"]>=9].copy()
doubles = []
for (i,row1),(j,row2) in itertools.combinations(singles.iterrows(),2):
    if row1["MatchID"]!=row2["MatchID"]:
        total_odds = row1["Odds"]*row2["Odds"]
        if total_odds>=90:
            doubles.append((row1,row2,total_odds))
len(doubles), doubles[:2]

## 4. Backtest flat £10 staking

In [ ]:
capital = 1000
stake = 10
history = []
for row in singles.itertuples():
    outcome = np.random.rand()<row.ModelProb
    if outcome:
        profit = (row.Odds-1)*stake
    else:
        profit = -stake
    capital+=profit
    history.append(capital)
plt.plot(history)
plt.title("Equity curve (simulated)")
plt.show()

## 5. Placeholder: Upload real odds CSV
Upload a file `exact_score_odds.csv` with columns:
```
Date,HomeTeam,AwayTeam,Score,DecimalOdds
2025-09-01,Arsenal,Chelsea,1-0,9.0
2025-09-01,Arsenal,Chelsea,2-1,12.0
...
```
Then run the cell below.

In [ ]:
try:
    from google.colab import files
    uploaded = files.upload()
    real_odds = pd.read_csv(list(uploaded.keys())[0])
    display(real_odds.head())
except:
    print("Not running in Colab - skip upload step")

## 6. ML demo with LightGBM
We simulate training an ML model for predicting goals.

In [ ]:
X = np.random.rand(200,5)
y = np.random.poisson(1.4,200)
dtrain = lgb.Dataset(X, label=y)
params = {"objective":"poisson","metric":"rmse"}
bst = lgb.train(params, dtrain, 50)
preds = bst.predict(np.random.rand(10,5))
preds